<a href="https://colab.research.google.com/github/joaowinderfeldbussolotto/assistente-ppc-ciencia-da-computacao/blob/main/tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU langchain-pinecone pinecone-notebooks openparse[ml] html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 126.2 kB/s eta 0:00:00
   ━━━━

## Configuração de Variaveis

In [8]:
from google.colab import userdata

class Settings:
  HF_TOKEN          = userdata.get('HF_TOKEN')
  PINECONE_API_KEY  = userdata.get('PINECONE_API_KEY')
  GROQ_API_KEY      = userdata.get('GROQ_API_KEY')


settings = Settings()

## Pré-processamento de dados

### Download do dataset

In [9]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc_2018.pdf"

--2024-08-25 21:45:36--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1383621 (1.3M) [application/pdf]
Saving to: ‘data/ppc_2018.pdf’

data/ppc_2018.pdf   100%[===================>]   1.32M   710KB/s    in 1.9s    

2024-08-25 21:45:40 (710 KB/s) - ‘data/ppc_2018.pdf’ saved [1383621/1383621]



In [10]:
from google.colab import files

uploaded = files.upload()

Saving PPC_ABRIL-22(nova versão)-v35.pdf to PPC_ABRIL-22(nova versão)-v35.pdf




### Transformação PDF -> HTML -> Markdown




In [148]:
import openparse
import html2text


def parse_document(file_path):

  parser = openparse.DocumentParser(table_args={"parsing_algorithm": "pymupdf"})
  return parser.parse(file_path)

def get_html_content(document):
  html_content = ''

  for node in document.nodes:
      html_content += node.text

  return html_content

def html_to_markdown_file(html: str, output_file: str) -> None:
    converter = html2text.HTML2Text()

    converter.ignore_links = False
    converter.ignore_images = False
    converter.body_width = 0

    markdown = converter.handle(html)

    with open(output_file, 'w') as file:
        file.write(markdown.strip())


In [ ]:
parsed_document = parse_document('data/ppc_2018.pdf')
html_content = get_html_content(parsed_document)
html_to_markdown_file(html_content, 'data/ppc_2018.md')

In [149]:
parsed_document = parse_document('data/PPC_ABRIL-22(nova versão)-v35.pdf')
html_content = get_html_content(parsed_document)
html_to_markdown_file(html_content, 'data/ppc_2024.md')

### Geração de embeddings e indíces do banco vetorial

In [13]:
class EmbeddingModelSpecs:
  def __init__(self):
    self.name      = 'sentence-transformers/distiluse-base-multilingual-cased-v1'
    self.dimension = 512

embeddings_model = EmbeddingModelSpecs()


In [14]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=settings.PINECONE_API_KEY)

In [15]:
import time

index_name = "index-ppc-2018-markdown"
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=embeddings_model.dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [16]:
!pip install -qU langchain-huggingface langchain langchain-community unstructured==0.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00


In [17]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=embeddings_model.name)

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [18]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [19]:
from uuid import uuid4
from langchain_core.documents import Document
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_documents(file_path):
    """Load documents from a Markdown file using UnstructuredMarkdownLoader."""
    # Usar um loader específico para Markdown
    loader = UnstructuredMarkdownLoader(file_path)
    documents = loader.load()
    return documents

def split_documents_into_chunks(documents, chunk_size=2000, chunk_overlap=400):
    """Split the loaded documents into text chunks based on Markdown structure."""
    # Usando RecursiveCharacterTextSplitter para respeitar a estrutura do Markdown
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n# ", "\n## ", "\n### ", "\n", " "]
    )
    texts = text_splitter.split_documents(documents)
    return texts

def create_documents_from_chunks(texts, source_name):
    """Create Document objects from text chunks."""
    documents = [
        Document(page_content=text.page_content, metadata={"source": source_name}) for text in texts
    ]
    return documents

def generate_unique_ids(num_ids):
    """Generate a list of unique UUIDs."""
    return [str(uuid4()) for _ in range(num_ids)]

def add_documents_to_vector_store(vector_store, documents, uuids):
    """Add documents with unique IDs to the vector store."""
    print(vector_store.add_documents(documents=documents, ids=uuids))


### Ingestão PPC 2018

In [83]:
file_path = "data/ppc_2018.md"
text = load_documents(file_path)
chunks = split_documents_into_chunks(text)
documents = create_documents_from_chunks(chunks, source_name="ppc2018.md")
uuids = generate_unique_ids(len(documents))

In [85]:
add_documents_to_vector_store(vector_store, documents, uuids)

['5b95dc3e-3b5c-464b-b982-6ccca2ceae83', '7b7d4aa2-8be7-4637-970e-646094af6788', 'a54ebfb0-a8f6-4a31-985b-bbd6904691af', '733cebd4-6239-4a3a-9c5b-97b0b07d5a62', 'd05ac8b8-8c85-490a-9bda-793d1d60bc97', '9de7184b-4746-4bf3-9a91-f92a25b8d8d1', '7384cdb9-5daa-4a64-a762-06d3d2cb2127', '3632d458-fc31-4e63-941e-61d1da3f0ad2', '80b3a241-f8ad-4bd6-b30d-ba0141468e89', 'c00c2861-36fd-4c57-bc92-799d8051be2c', '08ac0930-7335-4266-82ed-30dea1919b1d', '0d3730a0-f2e1-4099-85ac-6d5c65909382', '55b31d92-28fb-4694-b139-329643ec0c6d', 'bcc3a08f-3079-4d63-92dd-1e4330f43f0b', '5fdf5e3a-da59-41f1-b25d-06f2919ac7d8', '420b30d0-042b-4d24-9f17-4806e5dbbe4c', '3cd1506e-4c45-4588-a9af-a1f63af10d05', 'e300c54e-e61e-4763-a32f-392fe44f4874', 'c83dc7cc-8f4b-4a8b-a069-cfae2536f500', 'd5fe843e-a606-4791-972a-14abbf60b11e', '28433d44-88f8-426e-8271-6a7c08b0a376', 'f2906bda-1a74-460b-8644-e64b13c97ee8', '0b031fa4-7fc8-48fd-92c6-67a7bbf9fd14', 'aeac33be-8c61-47b8-9ea2-439b7c89ed40', '5681aad3-92f6-4cb8-bd3e-6e0c7748e0d6',

### Ingestão PPC 2024

In [150]:
file_path = "data/ppc_2024.md"
text = load_documents(file_path)
chunks = split_documents_into_chunks(text)
documents = create_documents_from_chunks(chunks, source_name="ppc2024.md")
uuids = generate_unique_ids(len(documents))

In [151]:
add_documents_to_vector_store(vector_store, documents, uuids)

['d7390c7c-5fc5-4335-ac10-c573508bdab0', '7371ba19-a828-4b12-8abb-4a2e51aa81e5', 'cebcab7b-bcb8-4134-9d6f-03474ac04b17', '57c0c364-ad88-459a-9eb8-19a75876c9e1', '6eac18fc-c54f-4235-9c17-2adaa1fede84', '9c7611a3-c103-4e92-8a1f-80919821ddb6', '83dfb4d8-dab9-455d-abc9-a2d4143db12e', 'bd726e25-e7c7-447f-92fb-2779593bde89', '83fdc6d5-74e8-4a86-a3a5-b31b1c938b25', 'a6bf392b-f2e6-4f96-831c-556221d2a5a6', 'b9a42332-297a-4e40-a6d6-283fd556b296', '906fa7ec-bb87-43ec-b65b-a0deca06f782', '2fc66fc0-8819-44b9-81ff-4f1bca6f6078', '79023b05-80b2-486c-b231-d8b257bf3f3f', '6262a2a4-82f6-4942-b580-9e0ed60eb1f4', 'e35a39a6-25da-4d0f-9c36-a1d16d09920c', '26ac4c31-8d97-4728-828f-01fa9593f1ad', 'c8196ad0-725c-431e-8f60-8b027d1e470d', '637cad20-8e88-4d13-a6ac-e0517444ea9a', 'efa99a51-cc7e-4293-9cd5-0fcdef8ccc42', '4d7f1f38-adaf-4cdc-b60e-df64b20cdc8c', 'e3e072b9-15b1-42f0-9803-4807d4c71423', 'd05998ea-f427-4c83-8d2b-8a64e750fe10', 'eafbd1d0-bf03-4ab7-b511-7ca4519fc192', '5a8108e0-489c-4651-8fe2-8513c2919576',

In [146]:
# vector_store.delete(ids=uuids)



In [200]:
chunks[0]

Document(metadata={'source': 'data/ppc_2024.md'}, page_content='IDENTIFICAÇÃO INSTITUCIONAL\n\nA Universidade Federal da Fronteira Sul foi criada pela Lei Nº 12.029, de 15 de\n\n35 setembro de 2009. Tem abrangência interestadual com sede na cidade catarinense de\n\nChapecó, trêscampino Rio Grande do Sul – Cerro Largo, Erechim e Passo Fundo – e dois\n\ncampino Paraná – Laranjeiras do Sul e Realeza. Reitor: Marcelo Recktenvald\n\n45 Vice-Reitor: Gismael Francisco Perin\n\nPró-Reitor de Graduação: Jeferson Saccol Ferreira\n\nPró-Reitor de Pesquisa e Pós-Graduação: Clevison Luiz Giacobbo\n\nPró-Reitor de Extensão e Cultura:Patricia Romagnolli\n\nPró-Reitor de Administração e Infraestrutura:Rosangela Frassão Bonfanti\n\n50 Pró-Reitor de Planejamento: Everton Miguel da Silva Loreto\n\nPró-Reitor de Assuntos Estudantis:Rubens Fey\n\nPró-Reitor de Gestão de Pessoas:Claunir Pavan Dirigentes de Chapecó (SC)\n\n55 Diretor deCampus: Roberto Mauro Dallagnol\n\nCoordenador Administrativo: Diego de S

## Geração da resposta

In [23]:
!pip install -U langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.8 MB/s eta 0:00:00


In [183]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    groq_api_key=settings.GROQ_API_KEY,
    max_retries=2
)



def query_expansion(query):

    query_expansion_llm = ChatGroq(
        model="gemma2-9b-it",
        temperature=0,
        groq_api_key=settings.GROQ_API_KEY,
        max_retries=2
    )

    prompt = f"""Esse é minha pergunta: {query}
    Me retorne uma string com 5 jeitos de escrever essa mesma pergunta em portugues.
    Considere um contexto academico de uma universidade.
    Retorne apenas as strings separadas com virgula sem nenhuma frase introdutória.
    """

    # Criando o prompt para a consulta expandida
    prompt_template = ChatPromptTemplate.from_messages([("human", prompt)])

    # Gerando a lista de consultas expandidas
    chain = prompt_template | query_expansion_llm
    response = chain.invoke({})
    return response.content


def get_context(query, source_name = 'ppc2024.md'):

    results = vector_store.search(
        query,
        "similarity",
        k=5,
        filter={"source": source_name},
    )


    return  '\n'.join([result.page_content for result in results])




def build_prompt(contexto, pergunta):
    # Cria o prompt para o contexto do PPC
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Você é um assistente especializado no Plano Pedagógico do Curso (PPC) de Ciência da Computação da UFFS. "
                "Sua tarefa é fornecer respostas precisas sobre qualquer questão relacionada ao PPC, utilizando o contexto fornecido para guiar sua resposta."
                "Utilize apenas o contexto informado como referencia"
                "Caso a resposta não está no contexto, informe que não sabe a resposta e recomende consultar o documento original."

                "Responda exclusivamente em portugues brasileiro."
            ),
            (
            "human",
            f"Dado o contexto: {contexto}\n\nPergunta: {pergunta}. Utilize apenas o contexto fornecido para responder"),
        ]
    )
    return prompt

def get_completion(prompt):
    chain = prompt | llm

    response = chain.invoke({})
    return response


def get_answer(query, source_name):
    expanded_query = query_expansion(query)
    print(expanded_query)
    context = get_context('Atividades de Ensino', source_name)
    prompt = build_prompt(context, query)
    response = get_completion(prompt)

    return context, response.content


In [184]:
query = "Qual é o limite de horas para Atividades de Ensino?"
context, answer = get_answer(query, "ppc2024.md")
answer

Qual o limite máximo de horas para atividades de ensino? ,  
Qual a carga horária máxima permitida para atividades de ensino? ,
Até quantas horas por semana um professor pode dedicar a atividades de ensino? ,
Existe um limite de horas para atividades de ensino na universidade? ,
Em que medida a carga horária de atividades de ensino é regulamentada? 



'Based on the provided context, there is no specific limit for the number of hours for "Atividades de Ensino" (Teaching Activities) mentioned. However, it is stated that the Atividades Curriculares Complementares (ACCs) have a maximum of 210 horas-aula (classroom hours) and are mandatory for the curriculum\'s completion. Additionally, the Trabalho de Conclusão de Curso (Final Project) has a variable workload depending on the chosen project.\n\nIn summary, the context provided does not specify a limit for Teaching Activities\' hours.'

In [185]:
context

'planejamento para o semestre e verificação da execução do planejamento, respectivamente.\n\nAs atividades inerentes a um determinado componente curricular são de inteira\n\nresponsabilidade do docente do componente. Essas atividades são verificadas formalmente em DRAFT\n\ndois momentos: na aprovação do plano de ensino, momento durante o qual o Colegiado verifica\n\n1540 a coerência do mesmo em relação ao perfil do egresso e ao que está previsto no PPC, e nas\n\navaliações internas do curso. A avaliação realizada pelos discentes ocorre semestralmente,\n\npreferencialmente no início da segunda metade do semestre. Os resultados são discutidos de\n\nforma ampla no Colegiado e ocorre devolutiva das avaliações aos docentes. Caso necessário,\n\nanálise conjunta especificamente com o docente e coordenação está prevista no processo de\n\n1545 gestão (e.g., quando ocorrem muitas avaliações negativas). Os resultados das avaliações externas do curso e da Comissão Própria de Avaliação (CPA) da ins

In [189]:
results = vector_store.similarity_search(
    "Limite de Atividades de ensino",
    k=2,
    filter={"source": "ppc2024.md"},
)
results

[Document(metadata={'source': 'ppc2024.md'}, page_content='realizada levando-se em consideração a assiduidade e o aproveitamento nos estudos.\n\nPara ser aprovado num componente curricular, excetuando-se os casos amparados em lei\n\ne os componentes curriculares cursados a distância, o discente deverá ter frequência mínima de DRAFT\n\n1500 75% (setenta e cinco por cento) nas atividades desenvolvidas, cabendo ao docente o registro da\n\nfrequência dos discentes. A verificação do aproveitamento nos estudos e do alcance dos\n\nobjetivos previstos nos planos de ensino, em cada componente curricular, será realizada por\n\nmeio da aplicação de diferentes instrumentos de avaliação (provas, trabalhos, seminários,\n\ntrabalhos em grupo, entre outros) e da concepção de avaliação, podendo o docente atribuir pesos\n\n1505 distintos aos diferentes instrumentos, devidamente explicitados no Plano de Ensino. No transcorrer do processo avaliativo, garantir-se-á oportunidades de recuperação da aprendiza